In [1]:
import matplotlib.pyplot as plt
import numpy as np
import json
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Conv2DTranspose, Input, concatenate
from tensorflow.keras.models import Model
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
import plotly.express as px
import pickle
from tensorflow.keras.models import model_from_json
from tensorflow.keras.models import load_model
from mylib import load_image, load_mask, get_tf_dataset, dataset_generator, find_images_with_min_unique_values

2025-04-05 17:05:42.996563: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-04-05 17:05:43.079797: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-05 17:05:43.083879: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2025-04-05 17:05:43.083897: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart d

In [2]:
# 1️⃣ Load and Preprocess Data
# Constants to define
IMG_SIZE = (256, 256)
TRAIN_DATASET_PATH = "../dataset/train/"
TRAIN_IMAGES_PATH = os.path.join(TRAIN_DATASET_PATH, "images")
TRAIN_MASKS_PATH = os.path.join(TRAIN_DATASET_PATH, "targets")  # Segmentation masks
TRAIN_LABELS_PATH = os.path.join(TRAIN_DATASET_PATH, "labels")  # JSON labels

TEST_DATASET_PATH = "../dataset/test/"
TEST_IMAGES_PATH = os.path.join(TEST_DATASET_PATH, "images")
TEST_MASKS_PATH = os.path.join(TEST_DATASET_PATH, "targets")  # Segmentation masks
TEST_LABELS_PATH = os.path.join(TEST_DATASET_PATH, "labels")  # JSON labels

BATCH_SIZE = 16  # Number of images to treat at once (Adjust depending on your RAM)
BUFFER_SIZE = 2799  # Total number of images to load from dataset



In [3]:
# Load and visualize a sample image and mask (mask is a 5-channel image with one-hot encoding)
a = load_mask(TRAIN_MASKS_PATH+"/hurricane-michael_00000486_post_disaster_target.png")
b = load_image(TRAIN_IMAGES_PATH+"/hurricane-michael_00000486_post_disaster.png")
print(a.shape)
plt.subplot(321)
plt.title("Post Disaster Image")
plt.imshow(b)
plt.axis('off')
plt.subplot(322)
plt.title("Background pixels")
plt.imshow(a[:,:,0])
plt.axis('off')
plt.subplot(323)
plt.title("No damage")
plt.imshow(a[:,:,1])
plt.axis('off')
plt.subplot(324)
plt.title("Minor damage")
plt.imshow(a[:,:,2])
plt.axis('off')
plt.subplot(325)
plt.title("Major damage")
plt.imshow(a[:,:,3])
plt.axis('off')
plt.subplot(326)
plt.title("Destroyed")
plt.imshow(a[:,:,4])
plt.axis('off')



NameError: name 'IMG_SIZE' is not defined

In [17]:
# Create a list of the images and masks to load (here we use only the _pre_disaster images and then when needed we rename the list to _post_disaster)
train_image_filenames = sorted([
    f for f in os.listdir(TRAIN_IMAGES_PATH) 
    if "_pre_disaster.png" in f and not f.startswith(".") and ":Zone.Identifier" not in f # Exclude Windows Zone Identifier files
])

test_image_filenames = sorted([
    f for f in os.listdir(TEST_IMAGES_PATH) 
    if "_pre_disaster.png" in f and not f.startswith(".") and ":Zone.Identifier" not in f
])

# Get datasets
train_dataset = get_tf_dataset(train_image_filenames, TRAIN_IMAGES_PATH, TRAIN_MASKS_PATH, BATCH_SIZE)
test_dataset = get_tf_dataset(test_image_filenames, TEST_IMAGES_PATH, TEST_MASKS_PATH, BATCH_SIZE)

In [18]:

# 2️⃣ Define U-Net Model
def unet_model(input_shape=(256, 256, 6), num_classes=5):
    inputs = Input(input_shape)

    # Encoder
    c1 = Conv2D(64, (3,3), activation='relu', padding='same')(inputs)
    c1 = Conv2D(64, (3,3), activation='relu', padding='same')(c1)
    p1 = MaxPooling2D((2,2))(c1)

    c2 = Conv2D(128, (3,3), activation='relu', padding='same')(p1)
    c2 = Conv2D(128, (3,3), activation='relu', padding='same')(c2)
    p2 = MaxPooling2D((2,2))(c2)

    c3 = Conv2D(256, (3,3), activation='relu', padding='same')(p2)
    c3 = Conv2D(256, (3,3), activation='relu', padding='same')(c3)
    p3 = MaxPooling2D((2,2))(c3)

    # Bottleneck
    c4 = Conv2D(512, (3,3), activation='relu', padding='same')(p3)
    c4 = Conv2D(512, (3,3), activation='relu', padding='same')(c4)

    # Decoder
    u5 = Conv2DTranspose(256, (2,2), strides=(2,2), padding='same')(c4)
    u5 = concatenate([u5, c3])
    c5 = Conv2D(256, (3,3), activation='relu', padding='same')(u5)
    c5 = Conv2D(256, (3,3), activation='relu', padding='same')(c5)

    u6 = Conv2DTranspose(128, (2,2), strides=(2,2), padding='same')(c5)
    u6 = concatenate([u6, c2])
    c6 = Conv2D(128, (3,3), activation='relu', padding='same')(u6)
    c6 = Conv2D(128, (3,3), activation='relu', padding='same')(c6)

    u7 = Conv2DTranspose(64, (2,2), strides=(2,2), padding='same')(c6)
    u7 = concatenate([u7, c1])
    c7 = Conv2D(64, (3,3), activation='relu', padding='same')(u7)
    c7 = Conv2D(64, (3,3), activation='relu', padding='same')(c7)

    outputs = Conv2D(num_classes, (1,1), activation='softmax')(c7)  # Multiclass segmentation output
    model = Model(inputs, outputs)
    return model



In [19]:
# Load the model
model = unet_model() 
model.load_weights("../models/unet_v8_multiclass_epoch_20.weights.h5")
print(model.summary())

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 256, 256, 6  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_45 (Conv2D)             (None, 256, 256, 64  3520        ['input_4[0][0]']                
                                )                                                                 
                                                                                                  
 conv2d_46 (Conv2D)             (None, 256, 256, 64  36928       ['conv2d_45[0][0]']              
                                )                                                           

In [21]:

for test_images, test_masks in test_dataset.take(1):  
    image = test_images[1, :, :, :]  # First image (All 6 channels)
    mask = test_masks[1, :, :, :]  # First mask (grayscale)


2025-04-05 16:45:59.341062: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:390] Filling up shuffle buffer (this may take a while): 260 of 2799
2025-04-05 16:46:09.309364: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:390] Filling up shuffle buffer (this may take a while): 503 of 2799
2025-04-05 16:46:19.314904: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:390] Filling up shuffle buffer (this may take a while): 748 of 2799
2025-04-05 16:46:26.708791: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:415] Shuffle buffer filled.


In [20]:
imno = 7

b1 = test_images[imno, :, :, :3]  # Take the first image
b2 = test_images[imno, :, :, 3:]  # Take the second image
a = model.predict(test_images[:, :, :, :])  # Predict on the first image
a = a[imno, :, :, :]  # Take the first channel (background)
c = test_masks[imno, :, :, :]  # Take the first mask (grayscale)


NameError: name 'test_images' is not defined